## Схема работы сервиса warehouse

#### Программы и компоненты которые участвуют в проекте

python 3.12.3  
mysql  Ver 8.0.43-0ubuntu0.24.04.2 for Linux on x86_64 ((Ubuntu))

## 1. Архитектура программы

#### 1.1 БД

![Полная схема бд](./photos/db.png)
```
CREATE TABLE `products` (  
  `id` int NOT NULL AUTO_INCREMENT,  
  `name` varchar(255) NOT NULL,  
  `category_id` int NOT NULL,  
  `sku` varchar(100) NOT NULL,  
  `base_price` decimal(10,2) NOT NULL,  
  `total_quantity` int NOT NULL DEFAULT '0',  
  `is_active` tinyint(1) DEFAULT '1',  
  `created_at` timestamp NULL DEFAULT CURRENT_TIMESTAMP,  
  `updated_at` timestamp NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,  
  PRIMARY KEY (`id`),  
  UNIQUE KEY `sku` (`sku`),  
  KEY `category_id` (`category_id`),  
  CONSTRAINT `products_ibfk_1` FOREIGN KEY (`category_id`) REFERENCES `categories` (`id`) ON DELETE RESTRICT  
) ENGINE=InnoDB AUTO_INCREMENT=6 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;
```

У нас есть главная таблица: products в ней хранятся  униакльные id товаров, категории товаров, название и тд. По этому id у нас таблицы связаны с это главной таблицей, например товар из product_attributes_thermocups должны быть связнаны с products и при это категория у в products у записи должна совпадать, то есть то же быть термокружки

```
CREATE TABLE `product_attributes_thermocups` (
  `product_id` int NOT NULL,
  `volume_ml` int NOT NULL,
  `color` varchar(100) NOT NULL,
  `brand` varchar(255) NOT NULL,
  `model` varchar(255) DEFAULT NULL,
  `is_hermetic` tinyint(1) NOT NULL,
  `material` varchar(100) DEFAULT NULL,
  PRIMARY KEY (`product_id`),
  CONSTRAINT `product_attributes_thermocups_ibfk_1` FOREIGN KEY (`product_id`) REFERENCES `products` (`id`) ON DELETE CASCADE
);
CREATE TABLE `categories` (
  `id` int NOT NULL AUTO_INCREMENT,
  `name` varchar(255) NOT NULL,
  `description` text,
  `created_at` timestamp NULL DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (`id`),
  UNIQUE KEY `name` (`name`)
);
CREATE TABLE `product_attributes_servers` (
  `product_id` int NOT NULL,
  `ram_gb` int NOT NULL,
  `cpu_model` varchar(255) NOT NULL,
  `cpu_cores` int NOT NULL,
  `hdd_size_gb` int DEFAULT NULL,
  `ssd_size_gb` int DEFAULT NULL,
  `form_factor` enum('Rack','Tower','Blade') NOT NULL,
  `manufacturer` varchar(255) NOT NULL,
  PRIMARY KEY (`product_id`),
  CONSTRAINT `product_attributes_servers_ibfk_1` FOREIGN KEY (`product_id`) REFERENCES `products` (`id`) ON DELETE CASCADE
);
CREATE TABLE `product_stocks` (
  `id` int NOT NULL AUTO_INCREMENT,
  `product_id` int NOT NULL,
  `warehouse_id` int NOT NULL,
  `quantity` int NOT NULL DEFAULT '0',
  PRIMARY KEY (`id`),
  KEY `product_id` (`product_id`),
  KEY `warehouse_id` (`warehouse_id`),
  CONSTRAINT `product_stocks_ibfk_1` FOREIGN KEY (`product_id`) REFERENCES `products` (`id`) ON DELETE CASCADE,
  CONSTRAINT `product_stocks_ibfk_2` FOREIGN KEY (`warehouse_id`) REFERENCES `warehouses` (`id`) ON DELETE CASCADE
);
CREATE TABLE `warehouses` (
  `id` int NOT NULL AUTO_INCREMENT,
  `name` varchar(255) NOT NULL,
  `address` text,
  `is_active` tinyint(1) DEFAULT '1',
  PRIMARY KEY (`id`)
);
```

#### 2.1 Код